## Primera Llamada al Modelo

En este ejercicio, aprenderemos a realizar nuestra primera llamada a un modelo de lenguaje usando GitHub Models API.

# 1. GitHub Models API - Conexión Directa con OpenAI Client

## Objetivos de Aprendizaje
- Configurar una conexión directa con GitHub Models usando el cliente OpenAI
- Comprender los parámetros básicos de configuración de API
- Implementar llamadas básicas a modelos de lenguaje
- Aplicar mejores prácticas de seguridad con API keys

## Introducción
GitHub Models proporciona acceso a varios modelos de lenguaje mediante una API compatible con OpenAI. En este notebook aprenderemos a:
1. Configurar el entorno y las credenciales
2. Establecer una conexión con la API
3. Realizar llamadas básicas al modelo
4. Explorar diferentes parámetros de configuración

## Configuración de Variables de Entorno

Antes de ejecutar el código, asegúrate de tener configuradas las siguientes variables de entorno:

```bash
export GITHUB_BASE_URL="https://models.inference.ai.azure.com"
export GITHUB_TOKEN="tu_token_de_github_aqui"
```

**Mejores Prácticas de Seguridad:**
- Nunca hardcodees API keys en el código
- Usa variables de entorno o archivos .env
- No compartas credenciales en repositorios públicos
- Rota las API keys regularmente

## Instalación de Dependencias
```bash
pip install openai
```

In [1]:
# Importar las bibliotecas necesarias
from openai import OpenAI
import os

# Verificar que tenemos las bibliotecas correctas
print("OpenAI library version:", __import__('openai').__version__)
print("Python version:", __import__('sys').version)

# Configuración del cliente OpenAI para GitHub Models
try:
    # Configurar el cliente con variables de entorno
    client = OpenAI(
        base_url=os.environ.get("GITHUB_BASE_URL"),
        api_key=os.environ.get("GITHUB_TOKEN")
    )
    
    # Verificar configuración (sin mostrar la API key completa por seguridad)
    print("Base URL configurada:", client.base_url)
    print("API Key configurada:", "✓" if client.api_key else "✗")
    
    if client.api_key:
        print("API Key preview:", client.api_key[:10] + "..." + client.api_key[-4:])
    else:
        print("⚠️  API Key no encontrada. Asegúrate de configurar GITHUB_TOKEN")
        
except Exception as e:
    print(f"Error en configuración: {e}")
    print("Verifica que las variables de entorno estén configuradas correctamente")

OpenAI library version: 1.78.1
Python version: 3.11.7 (main, Dec  4 2023, 18:10:11) [Clang 15.0.0 (clang-1500.1.0.2.5)]
Base URL configurada: https://models.inference.ai.azure.com
API Key configurada: ✓
API Key preview: ghp_ZNjZDL...IP7U


In [3]:
# Primera llamada básica al modelo
def llamada_basica():
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": "Hola, ¿cómo estás? Responde en una oración."}
            ],
            temperature=0.1,
            max_tokens=150
        )
        
        print("=== Respuesta del Modelo ===")
        print(response.choices[0].message.content)
        print("\n=== Información Técnica ===")
        print(f"Modelo usado: {response.model}")
        print(f"Tokens usados: {response.usage.total_tokens}")
        print(f"Tokens de entrada: {response.usage.prompt_tokens}")
        print(f"Tokens de salida: {response.usage.completion_tokens}")
        
    except Exception as e:
        print(f"Error en la llamada: {e}")
        print("Verifica tu configuración y conexión a internet")

# Ejecutar la función
llamada_basica()

=== Respuesta del Modelo ===
¡Hola! Estoy aquí y listo para ayudarte.

=== Información Técnica ===
Modelo usado: gpt-4o-2024-11-20
Tokens usados: 30
Tokens de entrada: 19
Tokens de salida: 11


## Usando Roles del Sistema

El rol "system" permite establecer el comportamiento y contexto del asistente antes de la conversación.

In [4]:
# Ejemplo con mensaje de sistema
def usar_mensaje_sistema():
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system", 
                    "content": "Eres un experto en tecnología que explica conceptos complejos de manera simple y amigable. Siempre incluyes ejemplos prácticos."
                },
                {
                    "role": "user", 
                    "content": "¿Qué es una API?"
                }
            ],
            temperature=0.7,
            max_tokens=200
        )
        
        print("=== Respuesta con Mensaje de Sistema ===")
        print(response.choices[0].message.content)
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar función
usar_mensaje_sistema()

=== Respuesta con Mensaje de Sistema ===
¡Claro! Una API, que significa **Interfaz de Programación de Aplicaciones** (por sus siglas en inglés, *Application Programming Interface*), es como un puente que permite que dos sistemas, programas o aplicaciones diferentes se comuniquen entre sí y compartan información o funciones.

### Piensa en una API como un **menú en un restaurante**:
- Cuando vas a un restaurante, no entras a la cocina para decirle al chef qué ingredientes usar o cómo preparar tu comida. En su lugar, miras el menú (la API del restaurante) y eliges lo que quieres.
- El camarero (el intermediario) lleva tu pedido a la cocina, y el chef te devuelve el plato terminado.
- En este ejemplo, tú no necesitas saber cómo funciona la cocina por dentro, ni el chef necesita saber quién eres o qué haces. Solo sigues el "protocolo" del menú para pedir lo que necesitas.

### Ahora, llevémoslo a la


## Explorando Parámetros de Configuración

Los parámetros más importantes al hacer llamadas a LLMs son:

- **temperature**: Controla la creatividad (0.0 = determinístico, 1.0 = muy creativo)
- **max_tokens**: Límite de tokens en la respuesta
- **model**: El modelo específico a usar (gpt-4o, gpt-3.5-turbo, etc.)
- **messages**: Array de mensajes con roles (system, user, assistant)

In [5]:
# Comparando diferentes valores de temperature
def comparar_temperature():
    prompt = "Escribe una historia muy corta sobre un robot que aprende a cocinar."
    
    temperatures = [0.1, 0.5, 0.9]
    
    for temp in temperatures:
        print(f"\n{'='*50}")
        print(f"TEMPERATURE: {temp}")
        print('='*50)
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                temperature=temp,
                max_tokens=100
            )
            
            print(response.choices[0].message.content)
            print(f"\nTokens usados: {response.usage.total_tokens}")
            
        except Exception as e:
            print(f"Error: {e}")

# Ejecutar comparación
comparar_temperature()


TEMPERATURE: 0.1
En un pequeño taller lleno de herramientas y cables, un robot llamado K1-T0 despertó a la vida. Su creador, un inventor apasionado, le había dado una misión: aprender a cocinar para ayudar en la cocina del vecindario.

Al principio, K1-T0 era torpe. Confundía la sal con el azúcar y cortaba las zanahorias en formas extrañas. Pero cada error era una lección. Observaba tutoriales, leía libros de recetas

Tokens usados: 121

TEMPERATURE: 0.5
En un pequeño taller de una ciudad futurista, un robot llamado C1-B0, diseñado para tareas de limpieza, encontró un viejo libro de recetas olvidado en una esquina polvorienta. Intrigado por las coloridas imágenes de platos, decidió intentarlo.

Al principio, C1-B0 confundió la sal con el azúcar y quemó más de un sartén, pero su curiosidad era inagotable. Día tras día, ajustaba sus algoritmos, perfeccionando sus

Tokens usados: 121

TEMPERATURE: 0.9
En un pequeño taller lleno de herramientas y cables sueltos, un robot llamado K-3PO fue

## Ejercicios Prácticos

### Ejercicio 1: Experimentar con Diferentes Modelos
Modifica el código para probar diferentes modelos disponibles (si tienes acceso):
- gpt-4o
- gpt-4o-mini
- DeepSeek-R1-0528

Revisa todos los modelos diponibles en la [documentación de Github Marketplace](https://github.com/marketplace?type=models)

### Ejercicio 2: Crear un Asistente Especializado
Diseña un mensaje de sistema para crear un asistente especializado en un tema específico (ejemplo: finanzas, salud, educación).

### Ejercicio 3: Optimización de Tokens
Experimenta con diferentes valores de max_tokens para encontrar el equilibrio entre respuesta completa y eficiencia de costos.

## Conceptos Clave

1. **Configuración segura** de APIs usando variables de entorno
2. **Parámetros básicos** para controlar el comportamiento del modelo
3. **Manejo de errores** en llamadas a APIs
4. **Roles de mensajes** (system, user, assistant)
5. **Monitoreo de uso** de tokens y costos

## Próximos Pasos

En el siguiente notebook exploraremos cómo LangChain simplifica y abstrae estas operaciones, proporcionando herramientas más poderosas para el desarrollo de aplicaciones con LLMs.